In [1]:
import dask.dataframe as dd
import pyarrow

In [2]:
!ls /p/gpfs1/iopp/wisio_logs/_bottlenecks

genome_20240325_s10_proc_name	     recorder_genome_pegasus_32_15_30_45_60
genome_20240325_s45_proc_name	     recorder_genome_pegasus_32_mp
recorder_cm1_32_4		     recorder_genome_pegasus_32_mp_bhvr
recorder_flash_sedov3d_hdf5	     recorder_genome_pegasus_32_root
recorder_genome_pegasus_20240311s15  recorder_hacc_32_0
recorder_genome_pegasus_20240313s15  recorder_lbann_cosmoflow_20240321
recorder_genome_pegasus_20240314s15  recorder_lbann_cosmoflow_32
recorder_genome_pegasus_20240314s5   recorder_lbann_jag_20240321
recorder_genome_pegasus_32_15	     recorder_lbann_jag_32
recorder_genome_pegasus_32_15_30     recorder_montage_pegasus_32
recorder_genome_pegasus_32_15_30_45  recorder_mummi_wemul_32_32


In [4]:
ddf = dd.read_parquet('/p/gpfs1/iopp/wisio_logs/_bottlenecks/recorder_genome_pegasus_32_mp', engine='pyarrow')
ddf

,subject,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,metadata_size,read_count,write_count,metadata_count,read_min,read_max,write_min,write_max,metadata_min,metadata_max,data_count,data_size,data_time,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count,close_time,open_time,seek_time,stat_time,close_count,open_count,seek_count,stat_count,time_per,count_per,iops_slope,iops_bin,iops_score,time_overall,num_file_name,num_proc_name,num_time_range,metric,view_depth,view_name,excessive_metadata_access,excessive_metadata_access.reason.0,excessive_metadata_access.reason.1,excessive_metadata_access.reason.2,operation_imbalance,operation_imbalance.reason.0,operation_imbalance.reason.1,random_operations,random_operations.reason.0,size_imbalance,size_imbalance.reason.0,size_imbalance.reason.1,small_reads,small_reads.reason.0,small_reads.reason.1,small_writes,small_writes.reason.0,small_writes.reason.1
npartitions=46,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,float64,int64,int64,int64,int64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,int64,object,float64,float64,float64,float64,object,int64,object,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
ddf.memory_usage(deep=True).sum().compute()

5608829654

In [12]:
5608829654 / 1024 ** 3

5.2236296739429235

In [9]:
view_names = list(ddf['view_name'].unique().compute())
view_names

['time_range',
 'proc_name',
 'file_name',
 'time_range.proc_name',
 'time_range.file_name',
 'proc_name.time_range',
 'proc_name.file_name',
 'file_name.time_range',
 'file_name.proc_name',
 'time_range.proc_name.file_name',
 'time_range.file_name.proc_name',
 'proc_name.time_range.file_name',
 'proc_name.file_name.time_range',
 'file_name.time_range.proc_name',
 'file_name.proc_name.time_range']

In [48]:
root_view_names = list(ddf[ddf['view_depth'] == 1]['view_name'].unique().compute())
root_view_names

['time_range', 'proc_name', 'file_name']

In [7]:
reason_cols = ddf.columns[ddf.columns.str.contains('reason')]
reason_cols

Index(['excessive_metadata_access.reason.0',
       'excessive_metadata_access.reason.1',
       'excessive_metadata_access.reason.2', 'operation_imbalance.reason.0',
       'operation_imbalance.reason.1', 'random_operations.reason.0',
       'size_imbalance.reason.0', 'size_imbalance.reason.1',
       'small_reads.reason.0', 'small_reads.reason.1', 'small_writes.reason.0',
       'small_writes.reason.1'],
      dtype='object')

In [8]:
reason_rules = [(reason.split('.')[0], reason) for reason in reason_cols]
reason_rules

[('excessive_metadata_access', 'excessive_metadata_access.reason.0'),
 ('excessive_metadata_access', 'excessive_metadata_access.reason.1'),
 ('excessive_metadata_access', 'excessive_metadata_access.reason.2'),
 ('operation_imbalance', 'operation_imbalance.reason.0'),
 ('operation_imbalance', 'operation_imbalance.reason.1'),
 ('random_operations', 'random_operations.reason.0'),
 ('size_imbalance', 'size_imbalance.reason.0'),
 ('size_imbalance', 'size_imbalance.reason.1'),
 ('small_reads', 'small_reads.reason.0'),
 ('small_reads', 'small_reads.reason.1'),
 ('small_writes', 'small_writes.reason.0'),
 ('small_writes', 'small_writes.reason.1')]

In [25]:
reason_sets = {}
for rule, reason in reason_rules:
    reason_sets[rule] = reason_sets.get(rule, [])
    reason_sets[rule].append(reason)
reason_sets

{'excessive_metadata_access': ['excessive_metadata_access.reason.0',
  'excessive_metadata_access.reason.1',
  'excessive_metadata_access.reason.2'],
 'operation_imbalance': ['operation_imbalance.reason.0',
  'operation_imbalance.reason.1'],
 'random_operations': ['random_operations.reason.0'],
 'size_imbalance': ['size_imbalance.reason.0', 'size_imbalance.reason.1'],
 'small_reads': ['small_reads.reason.0', 'small_reads.reason.1'],
 'small_writes': ['small_writes.reason.0', 'small_writes.reason.1']}

In [28]:
reason_numbers = {rule: len(reason_sets[rule]) for rule in reason_sets}
reason_numbers

{'excessive_metadata_access': 3,
 'operation_imbalance': 2,
 'random_operations': 1,
 'size_imbalance': 2,
 'small_reads': 2,
 'small_writes': 2}

In [29]:
list(enumerate(reason_numbers))

[(0, 'excessive_metadata_access'),
 (1, 'operation_imbalance'),
 (2, 'random_operations'),
 (3, 'size_imbalance'),
 (4, 'small_reads'),
 (5, 'small_writes')]

In [36]:
reason_conditions = [(rule, " & ".join([f"~df['{rule}.reason.{i}']" for i in range(reason_numbers[rule])])) for rule in reason_numbers]
reason_conditions

[('excessive_metadata_access',
  "~df['excessive_metadata_access.reason.0'] & ~df['excessive_metadata_access.reason.1'] & ~df['excessive_metadata_access.reason.2']"),
 ('operation_imbalance',
  "~df['operation_imbalance.reason.0'] & ~df['operation_imbalance.reason.1']"),
 ('random_operations', "~df['random_operations.reason.0']"),
 ('size_imbalance',
  "~df['size_imbalance.reason.0'] & ~df['size_imbalance.reason.1']"),
 ('small_reads', "~df['small_reads.reason.0'] & ~df['small_reads.reason.1']"),
 ('small_writes',
  "~df['small_writes.reason.0'] & ~df['small_writes.reason.1']")]

In [13]:
df = ddf.compute()
df

,subject,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,...,random_operations.reason.0,size_imbalance,size_imbalance.reason.0,size_imbalance.reason.1,small_reads,small_reads.reason.0,small_reads.reason.1,small_writes,small_writes.reason.0,small_writes.reason.1
0,2427,0.037926,2,0,0,0,0.000000,0.000000,0.037926,0,...,False,False,False,False,False,False,True,False,False,True
1,3252,0.592998,66,0,0,0,0.000000,0.000000,0.592998,0,...,False,False,False,False,False,False,True,False,False,True
2,3255,0.528411,64,0,0,0,0.000000,0.000000,0.528411,0,...,False,False,False,False,False,False,True,False,False,True
3,3249,0.454279,60,0,0,0,0.000000,0.000000,0.454279,0,...,False,False,False,False,False,False,True,False,False,True
4,3253,0.368259,49,0,0,0,0.000000,0.000000,0.368259,0,...,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,1659,0.108917,66,0,16777216,134217728,0.108832,0.000000,0.047024,134217728,...,False,True,True,False,False,True,False,False,False,True
974,386,9.104730,5548,0,6436,422995,0.000000,0.002160,9.104730,0,...,False,True,False,True,False,False,True,False,False,True
975,2146,0.031084,19,16754824,16777216,67086472,0.031022,0.002332,0.000062,50331648,...,False,True,True,False,False,True,False,False,False,True
976,1494,0.086388,53,0,16777216,117439532,0.084624,0.001670,0.024885,100679680,...,False,True,True,False,False,True,False,False,False,True


In [20]:
df[(df['excessive_metadata_access'])]

,subject,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,...,random_operations.reason.0,size_imbalance,size_imbalance.reason.0,size_imbalance.reason.1,small_reads,small_reads.reason.0,small_reads.reason.1,small_writes,small_writes.reason.0,small_writes.reason.1
0,2427,0.037926,2,0,0,0,0.000000,0.000000,0.037926,0,...,False,False,False,False,False,False,True,False,False,True
1,3252,0.592998,66,0,0,0,0.000000,0.000000,0.592998,0,...,False,False,False,False,False,False,True,False,False,True
2,3255,0.528411,64,0,0,0,0.000000,0.000000,0.528411,0,...,False,False,False,False,False,False,True,False,False,True
3,3249,0.454279,60,0,0,0,0.000000,0.000000,0.454279,0,...,False,False,False,False,False,False,True,False,False,True
4,3253,0.368259,49,0,0,0,0.000000,0.000000,0.368259,0,...,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,242,10.089860,5905,0,5441,384040,0.000000,0.002226,10.089860,0,...,False,True,False,True,False,False,True,False,False,True
970,574,9.611187,5697,0,4942,329193,0.000000,0.001269,9.611187,0,...,False,True,False,True,False,False,True,False,False,True
971,571,8.899886,5335,0,6400,339844,0.000000,0.003276,8.899886,0,...,False,True,False,True,False,False,True,False,False,True
972,1009,4.393242,2658,0,16777216,5939525119,0.026156,0.001638,4.393242,5939134464,...,False,True,True,False,False,False,False,False,False,True


In [21]:
df[(df['excessive_metadata_access'] & df['excessive_metadata_access.reason.0'])]

,subject,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,...,random_operations.reason.0,size_imbalance,size_imbalance.reason.0,size_imbalance.reason.1,small_reads,small_reads.reason.0,small_reads.reason.1,small_writes,small_writes.reason.0,small_writes.reason.1
36,1928,2.904686,809,0,16777216,16942071,0.012850,0.003870,2.904686,16777216,...,False,True,True,False,False,False,True,False,False,True
38,1732,2.897765,1008,0,16777216,570524740,0.026609,0.003699,2.897765,570425344,...,False,True,True,False,False,False,True,False,False,True
39,1966,2.933564,1076,0,16777216,163530967,0.022578,0.976570,2.933564,100663296,...,False,True,True,False,False,False,True,False,False,True
41,1612,2.465891,970,0,16777216,503426955,0.004237,0.003704,2.465891,503316480,...,False,True,True,False,False,False,True,False,False,True
42,1971,2.623806,1090,0,16777216,176621738,0.002778,0.870305,2.623806,117440512,...,False,True,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,242,10.089860,5905,0,5441,384040,0.000000,0.002226,10.089860,0,...,False,True,False,True,False,False,True,False,False,True
970,574,9.611187,5697,0,4942,329193,0.000000,0.001269,9.611187,0,...,False,True,False,True,False,False,True,False,False,True
971,571,8.899886,5335,0,6400,339844,0.000000,0.003276,8.899886,0,...,False,True,False,True,False,False,True,False,False,True
972,1009,4.393242,2658,0,16777216,5939525119,0.026156,0.001638,4.393242,5939134464,...,False,True,True,False,False,False,False,False,False,True


In [24]:
df[(df['excessive_metadata_access'] & ~df['excessive_metadata_access.reason.0'] & ~df['excessive_metadata_access.reason.1'] & ~df['excessive_metadata_access.reason.2'])]

,subject,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,...,random_operations.reason.0,size_imbalance,size_imbalance.reason.0,size_imbalance.reason.1,small_reads,small_reads.reason.0,small_reads.reason.1,small_writes,small_writes.reason.0,small_writes.reason.1
1,3252,0.592998,66,0,0,0,0.0,0.0,0.592998,0,...,False,False,False,False,False,False,True,False,False,True
2,3255,0.528411,64,0,0,0,0.0,0.0,0.528411,0,...,False,False,False,False,False,False,True,False,False,True
3,3249,0.454279,60,0,0,0,0.0,0.0,0.454279,0,...,False,False,False,False,False,False,True,False,False,True
4,3253,0.368259,49,0,0,0,0.0,0.0,0.368259,0,...,False,False,False,False,False,False,True,False,False,True
5,3224,0.509311,71,0,0,0,0.0,0.0,0.509311,0,...,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103883,./tmphy492i8j,0.000943,1,0,0,0,0.0,0.0,0.000943,0,...,False,False,False,False,False,False,True,False,False,True
104618,./tmp4qycnzmi,0.000933,1,0,0,0,0.0,0.0,0.000933,0,...,False,False,False,False,False,False,True,False,False,True
376,2427,0.037823,1,0,0,0,0.0,0.0,0.037823,0,...,False,False,False,False,False,False,True,False,False,True
0,pegasus-kickstart#lassen331#87818#35184372401888,7.904303,1,0,0,0,0.0,0.0,7.904303,0,...,False,False,False,False,False,False,True,False,False,True


In [ ]:
unreason_count = sum(
        len(view_bot[(view_bot[rule] & eval(condition))])
        for rule, condition in reason_conditions
    )

In [41]:
for rule, condition in reason_conditions:
    print(len(df[df[rule] & eval(condition)]))
#     print(eval(condition))

18402
0
0
0
0
0


In [46]:
total_bot = 0
total_reason = 0
total_unreason = 0
for view_name in view_names:
    view_bot = df[df['view_name'] == view_name]
    bot_count = len(view_bot)
    total_bot = total_bot + bot_count
    reason_count = sum(
        len(view_bot[view_bot[rule] & view_bot[reason]])
        for rule, reason in reason_rules
    )
    unreason_count = sum(
        len(view_bot[view_bot[rule] & eval(condition.replace('df[', 'view_bot['))])
        for rule, condition in reason_conditions
    )
    total_reason = total_reason + reason_count
    total_unreason = total_unreason + unreason_count
    print(view_name, bot_count, reason_count, unreason_count)

time_range 1933 4621 37
proc_name 857 2369 46
file_name 6025052 6382744 7350
time_range.proc_name 1264 2562 1
time_range.file_name 359690 533553 4956
proc_name.time_range 1602 4038 0
proc_name.file_name 1690144 2006726 2319
file_name.time_range 1283 2758 38
file_name.proc_name 1263 2481 24
time_range.proc_name.file_name 115550 181170 2664
time_range.file_name.proc_name 1117 2123 0
proc_name.time_range.file_name 106504 207950 964
proc_name.file_name.time_range 1274 3010 1
file_name.time_range.proc_name 1393 2872 2
file_name.proc_name.time_range 978 2156 0


In [47]:
print('total_bot', total_bot)
print('total_reason', total_reason)
print('total_unreason', total_unreason)

total_bot 8309904
total_reason 9341133
total_unreason 18402


In [49]:
total_bot = 0
total_reason = 0
total_unreason = 0
for view_name in root_view_names:
    view_bot = df[df['view_name'] == view_name]
    bot_count = len(view_bot)
    total_bot = total_bot + bot_count
    reason_count = sum(
        len(view_bot[view_bot[rule] & view_bot[reason]])
        for rule, reason in reason_rules
    )
    unreason_count = sum(
        len(view_bot[view_bot[rule] & eval(condition.replace('df[', 'view_bot['))])
        for rule, condition in reason_conditions
    )
    total_reason = total_reason + reason_count
    total_unreason = total_unreason + unreason_count
    print(view_name, bot_count, reason_count, unreason_count)

time_range 1933 4621 37
proc_name 857 2369 46
file_name 6025052 6382744 7350


In [50]:
print('total_bot', total_bot)
print('total_reason', total_reason)
print('total_unreason', total_unreason)

total_bot 6027842
total_reason 6389734
total_unreason 7433


In [52]:
total_bot/total_reason

0.9433635265568175

In [51]:
total_unreason/total_bot

0.0012331112859295251

In [53]:
100-(total_unreason/total_bot*100)

99.87668887140704

In [45]:
for view_name in view_names:
    view_bot = df[df['view_name'] == view_name]
    unreason_count = {rule: 
        len(view_bot[view_bot[rule] & eval(condition.replace('df[', 'view_bot['))])
        for rule, condition in reason_conditions}
    print(view_name, unreason_count)

time_range {'excessive_metadata_access': 37, 'operation_imbalance': 0, 'random_operations': 0, 'size_imbalance': 0, 'small_reads': 0, 'small_writes': 0}
proc_name {'excessive_metadata_access': 46, 'operation_imbalance': 0, 'random_operations': 0, 'size_imbalance': 0, 'small_reads': 0, 'small_writes': 0}
file_name {'excessive_metadata_access': 7350, 'operation_imbalance': 0, 'random_operations': 0, 'size_imbalance': 0, 'small_reads': 0, 'small_writes': 0}
time_range.proc_name {'excessive_metadata_access': 1, 'operation_imbalance': 0, 'random_operations': 0, 'size_imbalance': 0, 'small_reads': 0, 'small_writes': 0}
time_range.file_name {'excessive_metadata_access': 4956, 'operation_imbalance': 0, 'random_operations': 0, 'size_imbalance': 0, 'small_reads': 0, 'small_writes': 0}
proc_name.time_range {'excessive_metadata_access': 0, 'operation_imbalance': 0, 'random_operations': 0, 'size_imbalance': 0, 'small_reads': 0, 'small_writes': 0}
proc_name.file_name {'excessive_metadata_access': 23

In [8]:
root = ddf[ddf['view_depth'] == 1]
root

,subject,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,metadata_size,read_count,write_count,metadata_count,read_min,read_max,write_min,write_max,metadata_min,metadata_max,data_count,data_size,data_time,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count,close_time,open_time,seek_time,stat_time,close_count,open_count,seek_count,stat_count,time_per,count_per,iops_slope,iops_bin,iops_score,time_overall,num_file_name,num_proc_name,num_time_range,metric,view_depth,view_name,excessive_metadata_access,excessive_metadata_access.reason.0,excessive_metadata_access.reason.1,excessive_metadata_access.reason.2,operation_imbalance,operation_imbalance.reason.0,operation_imbalance.reason.1,random_operations,random_operations.reason.0,size_imbalance,size_imbalance.reason.0,size_imbalance.reason.1,small_reads,small_reads.reason.0,small_reads.reason.1,small_writes,small_writes.reason.0,small_writes.reason.1
npartitions=530,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,float64,int64,int64,int64,int64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,int64,object,float64,float64,float64,float64,object,int64,object,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
%%time
root['iops_score'].value_counts().compute()

CPU times: user 1min 33s, sys: 21.2 s, total: 1min 54s
Wall time: 45.4 s


low          4042863
medium       3346879
very low     2335750
high          587169
critical      430411
very high     258088
none          229822
trivial        49891
Name: iops_score, dtype: int64